In [ ]:
%config IPCompleter.greedy=True

#Welcome to the big QQQ notebook

In [39]:
#imports
import pandas as pd
import time

In [40]:
#import data
qqq_day = pd.read_csv('latest_data/QQQ_1D_AUG2820.csv')
qqq_5m = pd.read_csv('latest_data/QQQ_5M_AUG2820.csv')
qqq_30m = pd.read_csv('latest_data/QQQ_30M_AUG2820.csv')

#constants
PROFIT_THRESHOLD = 0.002
BIG_PROFIT_THRESHOLD = 0.015
DAYS_OF_THE_WEEK = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

#utilities
def convert_epoch_time_to_day_of_the_week(epoch_milliseconds):
    epoch = epoch_milliseconds / 1000
    return time.strftime("%A", time.gmtime(epoch_milliseconds))

In [48]:
#PROCESS DATA
def ProcessData(dat):
    for i, row in dat.iterrows():
        if i == 0:
            continue
        #add day
        dat.at[i,'dayofweek'] = convert_epoch_time_to_day_of_the_week(dat.at[i, 'time'])
        #add profitability
        if row['close'] / dat.at[i-1, 'close'] - 1 > PROFIT_THRESHOLD:
            dat.at[i,'profit'] = 1
        else:
            dat.at[i,'profit'] = 0
        if row['close'] / dat.at[i-1, 'close'] - 1 > BIG_PROFIT_THRESHOLD:
            dat.at[i,'bigprofit'] = 1
        else:
            dat.at[i,'bigprofit'] = 0   
    return dat

qqq_day = ProcessData(qqq_day)
qqq_30m = ProcessData(qqq_30m)
qqq_5m = ProcessData(qqq_5m)

In [59]:
#PROFITABILITY PROBABILITY PER DAY
#parameters
PB_DAY_WINDOW = 21

def GetProfitableDayCount(dat, day, window):
    daycnt = 0
    pcnt = 0
    dat_window = dat.tail(window)
    for i, row in dat_window.iterrows():
        if dat.at[i, 'dayofweek'] == day:
            daycnt += 1
            if dat.at[i, 'profit'] == 1:
                pcnt += 1
    return daycnt, pcnt, pcnt/daycnt

result_profit_probability_per_day = []
for day in DAYS_OF_THE_WEEK:
    result_profit_probability_per_day.append(GetProfitableDayCount(qqq_day, day, PB_DAY_WINDOW))
    
result_profit_probability_per_day

[(4, 3, 0.75), (4, 3, 0.75), (4, 3, 0.75), (4, 3, 0.75), (5, 3, 0.6)]

In [ ]:
#DAY TRADE OPPORTUNITIES PER DAY
EVENT_THRESHOLD = 0.004

def GetDayTradeCallOpportunities(dat):
    day = dat.at[0, 'dayofweek']
    days = 1
    events = []
    dcnt = 0
    for i, row in dat.iterrows():
        if day != dat.at[i, 'dayofweek']:
            low = row
            day = dat.at[i, 'dayofweek']  
            days += 1
            dcnt = 0
            continue
            
        dat.at[i, 'callopportunities'] = dcnt
        dat.at[i, 'day'] = days        
        if i == 0:
            low = row
            continue
        
        if dat.at[i, 'close'] < low:
            low = row
            continue
        
        pctchange = ((row['close'] - low[1]) / low[1])
        if pctchange > EVENT_THRESHOLD:
            events.append((low,row))
            dcnt += 1
            low = row
    return (dat, events)
        

In [ ]:
#EXAMINE WHAT LEADS TO BIG PROFITS

In [60]:
#EXAMINE RELATIONSHIPS WITH PRECIOUS METALS

In [ ]:
#EXAMINE RELTIONSHIPS WITH OTHER INDEXES